In [13]:
import os
import numpy as np 
import pandas as pd 
from sklearn import model_selection,preprocessing,impute

train_data=pd.read_csv('data/train.csv')
test_data=pd.read_csv('data/test.csv')

train_data.head()

,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,0,e,8.80,f,s,u,f,a,c,w,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,a
1,1,p,4.51,x,h,o,f,a,c,n,...,NaN,y,o,NaN,NaN,t,z,NaN,d,w
2,2,e,6.94,f,s,b,f,x,c,w,...,NaN,s,n,NaN,NaN,f,f,NaN,l,w
3,3,e,3.88,f,y,g,f,s,NaN,g,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
4,4,e,5.85,x,l,w,f,d,NaN,w,...,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a


In [14]:
train_data_dropped=train_data.copy()
train_data_dropped=train_data_dropped.drop(['id'],axis=1)
train_data_dropped.isna().sum()

class                         0
cap-diameter                  4
cap-shape                    40
cap-surface              671023
cap-color                    12
does-bruise-or-bleed          8
gill-attachment          523936
gill-spacing            1258435
gill-color                   57
stem-height                   0
stem-width                    0
stem-root               2757023
stem-surface            1980861
stem-color                   38
veil-type               2957493
veil-color              2740947
has-ring                     24
ring-type                128880
spore-print-color       2849682
habitat                      45
season                        0
dtype: int64

In [15]:
test_data_dropped=test_data.copy()
test_data_dropped=test_data_dropped.drop(['id'],axis=1)
test_data_dropped.isna().sum()

cap-diameter                  7
cap-shape                    31
cap-surface              446904
cap-color                    13
does-bruise-or-bleed         10
gill-attachment          349821
gill-spacing             839595
gill-color                   49
stem-height                   1
stem-width                    0
stem-root               1838012
stem-surface            1321488
stem-color                   21
veil-type               1971545
veil-color              1826124
has-ring                     19
ring-type                 86195
spore-print-color       1899617
habitat                      25
season                        0
dtype: int64

In [16]:
num_cols=train_data_dropped.select_dtypes(include=np.number).columns.tolist()

In [17]:
cat_cols=[col for col in train_data_dropped.columns if col not in num_cols and col!='class']

In [18]:
numeric_imputer=impute.KNNImputer()
cat_imputer=impute.SimpleImputer(strategy='constant')
train_data_dropped['cap-diameter']=numeric_imputer.fit_transform(train_data_dropped['cap-diameter'].values.reshape(-1,1))
test_data_dropped['cap-diameter']=numeric_imputer.fit_transform(test_data_dropped['cap-diameter'].values.reshape(-1,1))
train_data_dropped[cat_cols]=cat_imputer.fit_transform(train_data_dropped[cat_cols].values)
test_data_dropped[cat_cols]=cat_imputer.fit_transform(test_data_dropped[cat_cols].values)


In [19]:
train_data_dropped.isna().sum()

class                   0
cap-diameter            0
cap-shape               0
cap-surface             0
cap-color               0
does-bruise-or-bleed    0
gill-attachment         0
gill-spacing            0
gill-color              0
stem-height             0
stem-width              0
stem-root               0
stem-surface            0
stem-color              0
veil-type               0
veil-color              0
has-ring                0
ring-type               0
spore-print-color       0
habitat                 0
season                  0
dtype: int64

In [20]:
for col in cat_cols:
    freq = train_data_dropped[col].value_counts()/len(train_data_dropped)
    train_data_dropped[col] = train_data_dropped[col].map(freq)
    test_data_dropped[col] = test_data_dropped[col].map(freq)

In [21]:
train_data_dropped.shape,test_data_dropped.shape

((3116945, 21), (2077964, 20))

In [22]:
class_dict={'e':0,'p':1}
train_data_dropped['class']=train_data_dropped['class'].map(class_dict)

In [23]:
train_data_final,val_data=model_selection.train_test_split(train_data_dropped,test_size=0.1,stratify=train_data_dropped['class'])

In [24]:
train_data_final.to_csv('data/train_data_processed.csv',index=False)
val_data.to_csv('data/val_data_processed.csv',index=False)
test_data_dropped.to_csv('data/test_data_processed.csv',index=False)